In [9]:
import pandas as pd
import numpy as np
import os

In [10]:
cwd = os.getcwd()

In [11]:
bundesliga_dir = cwd + '/../data/matches/Bundesliga'
la_liga_dir = cwd + '/../data/matches/La Liga'
ligue_1_dir = cwd + '/../data/matches/Ligue 1'
premier_league_dir = cwd + '/../data/matches/Premier League'
serie_a_dir = cwd + '/../data/matches/Serie A'

In [12]:
matches = pd.read_csv(premier_league_dir + '/Season 2021-2022.csv', index_col=0)

In [13]:
matches = matches.drop(matches.iloc[:, 23:],axis = 1)
matches = matches.drop(['HTHG', 'HTAG', 'HTR', 'Referee'],axis = 1)
matches = matches.reset_index(drop=True)

In [14]:
matches['Datetime'] = matches.apply(lambda row: row.Date + ' ' + row.Time, axis=1)

In [15]:
matches = matches.drop(['Date', 'Time'],axis = 1)

In [16]:
matches['Datetime'] = pd.to_datetime(matches['Datetime'], format='%d/%m/%Y %H:%M')

In [17]:
matches = matches.rename(columns=str.lower)

In [18]:
# Shot conversion rate
matches['hscr'] = matches['fthg'] / matches['hs']
matches['ascr'] = matches['ftag'] / matches['as']

In [19]:
matches = matches.sort_values('datetime')

In [20]:
matches.head()

,hometeam,awayteam,fthg,ftag,ftr,hs,as,hst,ast,hf,af,hc,ac,hy,ay,hr,ar,datetime,hscr,ascr
0,Brentford,Arsenal,2,0,H,8,22,3,4,12,8,2,5,0,0,0,0,2021-08-13 20:00:00,0.250000,0.000000
1,Man United,Leeds,5,1,H,16,10,8,3,11,9,5,4,1,2,0,0,2021-08-14 12:30:00,0.312500,0.100000
2,Burnley,Brighton,1,2,A,14,14,3,8,10,7,7,6,2,1,0,0,2021-08-14 15:00:00,0.071429,0.142857
3,Chelsea,Crystal Palace,3,0,H,13,4,6,1,15,11,5,2,0,0,0,0,2021-08-14 15:00:00,0.230769,0.000000
4,Everton,Southampton,3,1,H,14,6,6,3,13,15,6,8,2,0,0,0,2021-08-14 15:00:00,0.214286,0.166667


In [21]:
previous_season_standing = {}

for index, match in matches.iterrows():
    if match['hometeam'] not in previous_season_standing:
        if match['ftr'] == 'H':
            score = 3
        elif match['ftr'] == 'A':
            score = 0
        else:
            score = 1
        previous_season_standing[match['hometeam']] = score
    else:
        if match['ftr'] == 'H':
            score = 3
        elif match['ftr'] == 'A':
            score = 0
        else:
            score = 1
        previous_season_standing[match['hometeam']] += score
    if match['awayteam'] not in previous_season_standing:
        if match['ftr'] == 'A':
            score = 3
        elif match['ftr'] == 'H':
            score = 0
        else:
            score = 1
        previous_season_standing[match['awayteam']] = score
    else:
        if match['ftr'] == 'A':
            score = 3
        elif match['ftr'] == 'H':
            score = 0
        else:
            score = 1
        previous_season_standing[match['awayteam']] += score

In [22]:
previous_season_standing = {k:v for k, v in sorted(previous_season_standing.items(), key=lambda item: item[1])}
previous_season_standing

{'Norwich': 22,
 'Watford': 23,
 'Burnley': 35,
 'Leeds': 38,
 'Everton': 39,
 'Southampton': 40,
 'Aston Villa': 45,
 'Brentford': 46,
 'Crystal Palace': 48,
 'Newcastle': 49,
 'Brighton': 51,
 'Wolves': 51,
 'Leicester': 52,
 'West Ham': 56,
 'Man United': 58,
 'Arsenal': 69,
 'Tottenham': 71,
 'Chelsea': 74,
 'Liverpool': 92,
 'Man City': 93}

In [23]:
X = []
y = []

for _, match in matches.iterrows():
    previous_matches = matches[matches['datetime'] < match['datetime']]
    previous_hometeam_matches = previous_matches[(previous_matches['hometeam'] == match['hometeam']) | (previous_matches['awayteam'] == match['hometeam'])]
    previous_awayteam_matches = previous_matches[(previous_matches['hometeam'] == match['awayteam']) | (previous_matches['awayteam'] == match['awayteam'])]
    if previous_hometeam_matches.shape[0] > 0 and previous_awayteam_matches.shape[0] > 0:
        hometeam_rolling_values = [] # Shots, Shots On Target, Corners, Foul Committed, Yellow Cards, Red Cards, Score Conversion Rate
        awayteam_rolling_values = []
        for _, previous_match in previous_hometeam_matches.iterrows():
            if previous_match['hometeam'] == match['hometeam']:
                hometeam_rolling_values.append([previous_match['hs'], previous_match['hst'], previous_match['hc'], previous_match['hf'], previous_match['hy'], previous_match['hr'], previous_match['hscr']])
            if previous_match['awayteam'] == match['hometeam']:
                hometeam_rolling_values.append([previous_match['as'], previous_match['ast'], previous_match['ac'], previous_match['af'], previous_match['ay'], previous_match['ar'], previous_match['ascr']])
        for _, previous_match in previous_awayteam_matches.iterrows():
            if previous_match['hometeam'] == match['awayteam']:
                awayteam_rolling_values.append([previous_match['hs'], previous_match['hst'], previous_match['hc'], previous_match['hf'], previous_match['hy'], previous_match['hr'], previous_match['hscr']])
            if previous_match['awayteam'] == match['awayteam']:
                awayteam_rolling_values.append([previous_match['as'], previous_match['ast'], previous_match['ac'], previous_match['af'], previous_match['ay'], previous_match['ar'], previous_match['ascr']])
        if len(hometeam_rolling_values) > 5:
            hometeam_rolling_values = hometeam_rolling_values[-6:]
        if len(awayteam_rolling_values) > 5:
            awayteam_rolling_values = awayteam_rolling_values[-6:]
        X.append(np.concatenate([np.mean(hometeam_rolling_values, axis=0), np.mean(awayteam_rolling_values, axis=0)]))
        if match['ftr'] == 'H':
            y.append(1)
        elif match['ftr'] == 'A':
            y.append(-1)
        else:
            y.append(0)
        X.append(np.concatenate([np.mean(awayteam_rolling_values, axis=0), np.mean(hometeam_rolling_values, axis=0)]))
        if match['ftr'] == 'H':
            y.append(-1)
        elif match['ftr'] == 'A':
            y.append(1)
        else:
            y.append(0)
        

In [24]:
X = np.array(X)
y = np.array(y)

In [25]:
from sklearn.ensemble import RandomForestClassifier

In [27]:
rf = RandomForestClassifier(n_estimators=50, min_samples_split=10, random_state=1)

In [28]:
rf.fit(X, y)

RandomForestClassifier(min_samples_split=10, n_estimators=50, random_state=1)

In [222]:
matches = pd.read_csv(premier_league_dir + '/Season 2019-2020.csv', index_col=0)

matches = matches.drop(matches.iloc[:, 23:],axis = 1)
matches = matches.drop(['HTHG', 'HTAG', 'HTR', 'Referee'],axis = 1)
matches = matches.reset_index(drop=True)

matches['Datetime'] = matches.apply(lambda row: row.Date + ' ' + row.Time, axis=1)
matches = matches.drop(['Date', 'Time'],axis = 1)
matches['Datetime'] = pd.to_datetime(matches['Datetime'], format='%d/%m/%Y %H:%M')

matches = matches.rename(columns=str.lower)

# Shot conversion rate
matches['hscr'] = matches['fthg'] / matches['hs']
matches['ascr'] = matches['ftag'] / matches['as']

In [224]:
correct = 0
incorrect = 0

for _, match in matches.iterrows():
    previous_matches = matches[matches['datetime'] < match['datetime']]
    previous_hometeam_matches = previous_matches[(previous_matches['hometeam'] == match['hometeam']) | (previous_matches['awayteam'] == match['hometeam'])]
    previous_awayteam_matches = previous_matches[(previous_matches['hometeam'] == match['awayteam']) | (previous_matches['awayteam'] == match['awayteam'])]
    if previous_hometeam_matches.shape[0] > 0 and previous_awayteam_matches.shape[0] > 0:
        hometeam_rolling_values = [] # Shots, Shots On Target, Corners, Foul Committed, Yellow Cards, Red Cards, Score Conversion Rate
        awayteam_rolling_values = []
        for _, previous_match in previous_hometeam_matches.iterrows():
            if previous_match['hometeam'] == match['hometeam']:
                hometeam_rolling_values.append([previous_match['hs'], previous_match['hst'], previous_match['hc'], previous_match['hf'], previous_match['hy'], previous_match['hr'], previous_match['hscr']])
            if previous_match['awayteam'] == match['hometeam']:
                hometeam_rolling_values.append([previous_match['as'], previous_match['ast'], previous_match['ac'], previous_match['af'], previous_match['ay'], previous_match['ar'], previous_match['ascr']])
        for _, previous_match in previous_awayteam_matches.iterrows():
            if previous_match['hometeam'] == match['awayteam']:
                awayteam_rolling_values.append([previous_match['hs'], previous_match['hst'], previous_match['hc'], previous_match['hf'], previous_match['hy'], previous_match['hr'], previous_match['hscr']])
            if previous_match['awayteam'] == match['awayteam']:
                awayteam_rolling_values.append([previous_match['as'], previous_match['ast'], previous_match['ac'], previous_match['af'], previous_match['ay'], previous_match['ar'], previous_match['ascr']])
        if len(hometeam_rolling_values) > 5:
            hometeam_rolling_values = hometeam_rolling_values[-6:]
        if len(awayteam_rolling_values) > 5:
            awayteam_rolling_values = awayteam_rolling_values[-6:]
        pred = rf.predict(np.array([np.concatenate([np.mean(hometeam_rolling_values, axis=0), np.mean(awayteam_rolling_values, axis=0)])]))
        if match['ftr'] == 'H' and pred == 1:
            correct += 1
        elif match['ftr'] == 'A' and pred == -1:
            correct += 1
        elif match['ftr'] == 'D' and pred == 0:
            correct += 1
        else:
            incorrect += 1
        pred = rf.predict(np.array([np.concatenate([np.mean(awayteam_rolling_values, axis=0), np.mean(hometeam_rolling_values, axis=0)])]))
        if match['ftr'] == 'H' and pred == -1:
            correct += 1
        elif match['ftr'] == 'A' and pred == 1:
            correct += 1
        elif match['ftr'] == 'D' and pred == 0:
            correct += 1
        else:
            incorrect += 1

In [225]:
correct / (correct + incorrect)

0.45